<a href="https://colab.research.google.com/github/paascorb/Pruebas-TFM/blob/main/HuggingFaceTraducciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pruebas traducciones HuggingFace

In [2]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.3 MB/s eta 0:00:00


In [4]:
from transformers import pipeline
model_checkpoint = "Helsinki-NLP/opus-mt-en-es"
translator = pipeline("translation", model=model_checkpoint)
text = "Hello world! Hugging Face is the best NLP tool."
translation = translator(text)

print(translation)

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Hola mundo! Hugging Face es la mejor herramienta NLP.'}]


In [5]:
translation = translator("It's well know that the lemons are juicy, I love lemons in my lemon tree. I have around 3.000 lemon trees.")

print(translation)

[{'translation_text': 'Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.'}]


Vemos que no termina de traducir todo, vamos a intentar hacer un programita para que traduzca todo:

Sacado de: https://stackoverflow.com/questions/68185061/strange-results-with-huggingface-transformermarianmt-translation-of-larger-tex

In [7]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import LineTokenizer
import math
import torch
import nltk
nltk.download('punkt')

if torch.cuda.is_available():  
  dev = "cuda"
else:  
  dev = "cpu" 
device = torch.device(dev)
 
mname = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
model.to(device)

lt = LineTokenizer()
batch_size = 8

text_short = "It's well know that the lemons are juicy, I love lemons in my lemon tree. I have around 3.000 lemon trees."
text_long = text_short * 30

paragraphs = lt.tokenize(text_long)   
translated_paragraphs = []

for paragraph in paragraphs:
    sentences = sent_tokenize(paragraph)
    batches = math.ceil(len(sentences) / batch_size)     
    translated = []
    for i in range(batches):
        sent_batch = sentences[i*batch_size:(i+1)*batch_size]
        model_inputs = tokenizer(sent_batch, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
        with torch.no_grad():
            translated_batch = model.generate(**model_inputs)
        translated += translated_batch
    translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    translated_paragraphs += [" ".join(translated)]

translated_text = "\n".join(translated_paragraphs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
translated_text

'Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los limones en mi limonero. Tengo alrededor de 3.000 limones.Es bien sabido que los limones son jugosos, me encantan los lim

In [11]:
def traducir_parrafos(parrafos, tokenizer, model, tam_bloque=8, ):
  parrafos_traducidos = []
  for parrafo in parrafos:
    frases = sent_tokenize(parrafo)
    batches = math.ceil(len(frases) / tam_bloque)     
    traducido = []
    for i in range(batches):

        bloque_enviado = frases[i*tam_bloque:(i+1)*tam_bloque]
        model_inputs = tokenizer(bloque_enviado, return_tensors="pt", 
                                 padding=True, truncation=True, 
                                 max_length=500).to(device)
        with torch.no_grad():
            bloque_traducido = model.generate(**model_inputs)
        traducido += bloque_traducido
    traducido = [tokenizer.decode(t, skip_special_tokens=True) for t in traducido]
    parrafos_traducidos += [" ".join(traducido)]
  return parrafos_traducidos

Vemos que tenemos resuelto este problema. Pero probemos un ejemplo con un texto que sea extenso pero tenga pocos signos de puntuación.

In [13]:
texto_largo = "That is an exagerated long text that intent to test our new and much usefull fuction to translate texts that not contain dots nor commas."

In [14]:
paragraphs = lt.tokenize(texto_largo)
traducir_parrafos(paragraphs, tokenizer, model)

['Es un texto largo y exagerado que intenta probar nuestra nueva y muy útil fucción para traducir textos que no contienen puntos ni comas.']

Vemos que funciona bien, tratemos ahora de alargarlo más:

In [15]:
texto_largo = "That is an exagerated long text that intent to test our new and much usefull fuction to translate texts that not contain dots nor commas and god cat cat dog horse frogs doves whales mouses."

In [16]:
paragraphs = lt.tokenize(texto_largo)
traducir_parrafos(paragraphs, tokenizer, model)

['Ese es un texto largo y exagerado que intenta probar nuestra nueva y muy útil fucción para traducir textos que no contienen puntos ni comas y dios gato gato perro ranas caballo palomas ballenas ratón.']

Lo vamos a dar por bueno.

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-es-en"
translator_rev = pipeline("translation", model=model_checkpoint)
text = "Muy buenas tardes, ¿Qué tal estás?."
translation = translator_rev(text)

print(translation)

[{'translation_text': 'Good afternoon, how are you?'}]
